In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [1]:
from trec.trecqrels import TrecQrels
from trec.trectopics import TrecTopics
from trec.treccorpus import TrecCorpus

In [2]:
from gensim.models import Doc2Vec

Load TREC doc2vec model

In [3]:
model_fname = "F:/Models/DM_2_doc2vec_trec_d1000_n5_w8_mc20_t12_e10_dm.model"

d2v = Doc2Vec.load(model_fname) 

vocab_size = len(d2v.wv.index2word)
vocab_dict = d2v.wv.vocab

doc_embedding_dim = d2v.vector_size

In [ ]:
# embedding_dim = 1000
# vocab_size = 146756
model = keras.Sequential([
    layers.Embedding(doc_embedding_dim, vocab_size, input_length=10),
    layers.Activation(activation='softmax')
])

model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit()

In [8]:
pname = "f:/Runs/topics/"
trectopics = TrecTopics(pname)
trectopics.init()

trectopics.vectorize(vocab_dict=vocab_dict, include_title=True, include_desc=True, include_narr=True)
# trectopics.get_topic_vector(topic_no=50)
# trectopics.init_vectors(vocab_dict)
trectopics.topics_vecs.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [10]:
# trectopics.vectorize(vocab_dict=vocab_dict, include_desc=True)

# temp = trectopics.topics_vecs.sum(axis=1)
temp2 = trectopics.get_topic_vector(51)
title2 = trectopics.get_title(51)
# for i in range(len(temp2)):
#     if temp2[i] is not 0:
#         print(temp2[i])

In [17]:
a=["report", "the"]
print([vocab_dict[word].index for word in a])

[170, 0]


In [8]:
# from gensim.utils import has_pattern
# print(has_pattern())
print(vocab_dict["discouragement"])

Vocab(count:210, index:52553, sample_int:4294967296)
